## Problem Set for Lecture II: Filtering and Aliasing (Bryan's Version)

Bryan Scott, CIERA/Northwestern

Presented at DSFP Session 20: Time Series Methods in Astronomy

Exercises adapted and expanded from Statistics, Data Mining, and Machine Learning in Astronomy, by Ivezić, Connolly, VanderPlas, and Gray

version 0.1, February 16, 2024

The goal of this notebook is to walk you through filtering and aliasing so that you can learn how to perform optimal filtering of an input signal and gain intuition for the impact that filtering has on a signal. The final part will help you build intuition for why aliasing occurs and what impact it has on your ability to characterize signals of interest. 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import scipy.fft as fft
import scipy.stats as stats

### Problem 1: Quick Simulated Data and Low Pass Filtering



### Part A) Simulated data and familiarizing yourself with the FFT

Let's begin by generating a mock time series consisting of a Gaussian signal that we'd like to measure and a large noise term. After you plot the signal, plot the FFT of the signal using the np.fft.fft function. When you plot, make sure you understand the structure and symmetries of the object FFT returns. We will discuss why this object behaves this way in Session 21 - it is related to deep facts about the Fourier transform and its discretization for running on digital processors. 

In [ ]:
# Create the noisy data

np.random.seed(5)
N = 2000
dt = 0.05
noise = 0.01

t =  #N evenly spaced samples dt apart 

sr =  # samples per interval 

freq =  # N * 1/dt, by hand

freq = # using convenience function in np.fft 

h1 = np.exp(-0.5 * ((t - 20.) / 1.0) ** 2) #gaussian signal
h2 = 0.3*np.sin(2*np.pi*0.5*t) # periodic signal 
hN1 = h1 +  # add noise to the first signal
hN2 = h2 +  #add noise to the second signal 


fig = plt.figure(figsize=(10, 6.75))


# First plot: noisy signal
ax = fig.add_subplot(211)
ax.plot(t, hN1, '-', c='gray', label = 'signal + noise')
ax.plot(t, h1, c = 'dodgerblue', label = 'signal')

ax.set_xlabel(r'$\lambda$')
ax.set_ylabel('flux')

ax = fig.add_subplot(212)
ax.plot(t, hN2, '-', c='gray', label = 'signal + noise')
ax.plot(t, h2, c = 'dodgerblue', label = 'signal')

# plot the FFTs

#plotting code goes here 

plt.xlabel('Freq (Hz)')
plt.ylabel('FFT Amplitude |X(freq)|')
plt.xlim(0, 10)

## Part b) Low Pass Filtering 

Write a function that returns 1 for frquencies below some critical value and 0 for values above that critical value. The numpy piecewise and lambda functions may be helpful, as before, an example of the syntax is in the documentation for numpy.piecewise. After filtering your signal from part a plot the original signal and the inverse transform of the filtered signal. Repeat this exercise for the different functions you plotted in part a. How is the recovered signal different? Repeat again for varying noise amplitudes.

In [ ]:
def low_pass(nu, nu_c):
    return 

freq = 

fig = plt.figure(figsize=(10, 6.75))

ax = fig.add_subplot(211)

ax.set_xlabel(r'$\lambda$')
ax.set_ylabel('flux')

# ax.plot(t, hN2, '-', c='gray', label = 'signal + noise')



ax.plot()
ax.plot(t, h2, label = 'signal', c = 'orange', ls = '-.')

ax = fig.add_subplot(212)

ax.plot()
ax.plot(t, h1, label = 'signal', c = 'orange', ls = '-.')

ax.set_xlim(0, 90)

ax.set_xlabel(r'$\lambda$')
ax.set_ylabel('flux')

plt.show()

### Problem 2: Optimal Filtering 

In the lecture, we discussed high pass and low pass filters. Another common filter you may encounter is the "Wiener Filter" which is an 'optimal filter' in the following sense. 

Suppose you have a signal you're interested in which we can call u(t). Rather than being a pure measurement of some underlying process, suppose u(t) is the product of a measurement made by some imperfect instrument with a response function u(t). The total observed signal is then the convolution of the signal we're interested in and the instrument response,

$$ 
s(t) = \int r(\tau) u(t-\tau) d\tau
$$

Let us also suppose, for complexity's sake, that the signal s(t) is not measured independent of noise contributions.That is, we want to infer u(t) from c(t), where 

$$
c(t) = s(t) + n(t) 
$$

where n(t) is some noise signal. How might we proceed? We want to find a filter that retains as much information from the original signal as possible while discarding information from the noise and deconvolving with respect to the instrument response. We can quantify our error in the least squares sense via

$$
\int | \tilde{u}(t) - u(t) |^2 dt = \int |\tilde{U}(f) - U(f)|^2 df 
$$

where the reconstructed signal is given by 

$$
\tilde{U}(f) = \frac{C(f) \Phi(f)}{R(f)}
$$

Minimizing the least squares error and plugging in all of the above expressions yields an 'optimal' filter:

$$
\Phi(f) = \frac{ |S(f)|^2 }{|S(f)|^2 + |N(f)|^2} 
$$

#### Part A) Why is this an optimal filter? Hint: In what limits does each term in the filter 'matter more'? 

#### Part B) Estimates of the Noise Power

Using the Wiener filter in practice requires you to have an estimate of the signal and noise spectra (and their sum) in advance. While this is theoretically problematic, in practice it's rarely an issue. We can proceed in two ways. A more sophisticated method is to define a signal and noise function with a functional form close to those expected in your simulated signal from part a. Then compute the filter and filter the total signal to recover the component of interest. Alternatively, it is often easier to just approximate the signal PSD as the total PSD of the observed signal and assume the noise is flat in frequency space. Try both. Some code for achieving this is outlined below.

In [ ]:
# compute the PSD of the input
N = len(t)
Df =  
f = fftpack.ifftshift(Df * (np.arange(N) - N / 2)) # shifts the central frequency to zero 

H = np.fft.fft()
PSD = 

def signal():  
    return 

def noise():
    return

# make some intelligent choices for 'guess' for each of the above parameters of your model functions:
# hint, what do all of the terms in the PSD tell you about the signal?

amp_guess = 
width_guess = 
signal_params = (amp_guess, width_guess)

noise_params = 

# now fit for the signal - noise:

def min_func(v): 
    # what should the minimum function return? What are you trying to minimize when you fit?
    return 

v0 = tuple(signal_params) + tuple(noise_params)
v = optimize.minimize(min_func, v0, method='Nelder-Mead')['x']

#### Part C) Implementing a Wiener Filter on a Noisy Signal 

Now implement both versions of the Wiener filter - based on the fit or directly from the PSD of the total observed signal and compare the results. 

In [ ]:
# P_S = signal(f, v[0], v[1])

P_S = 

P_N = 
Phi = 
Phi[0] = 1  # correct for DC offset

# Use Phi to filter and smooth the values
h_smooth = fftpack.ifft(Phi * H)

plt.plot(f, P_S)
plt.show()
plt.plot(f, P_N)
plt.show()

plt.plot(t, h_smooth)

### Problem 3: Sampling and Aliasing

We saw in the lecture that signals can be reconstructed if they are sampled at rates greater than the Nyquist frequency. What happens if we sample at rates that are 'too low'? The answer is that power from frequencies too low to be captured by a given sampling rate is misidentified as coming from frequencies that are measured. Let's consider two similar functions,

$$
f(t) = \cos\left(\frac{3 \pi}{2} t\right) 
$$

and 

$$
g(t) = \cos\left(\frac{\pi}{2} t\right)
$$

For different numbers of samples. 

### Part a) Starting in the Time Domain, plot the functions f(t) and g(t) over the interval (-4, 4) for n = 9 samples and n = 18 samples. From your plot, make sure you understand why undersampling leads to aliasing. Explain the origin of the term 'aliasing'. How did I pick this interval and this number of samples?

Hint: The word alias is defined as "an assumed or additional name that a person sometimes uses"

In [ ]:
t1 = 
t2 = 

plt.plot(t1, )
plt.plot(t1, )

plt.plot(t2, )
plt.plot(t2, )

# plt.grid()
plt.xlabel('t')
plt.ylabel('f(t), g(t)')
plt.legend(loc = 'upper right')

### Part B) Let's now plot the Fourier Transforms of each signal. 

In [ ]:
N1 = 
N2 = 

t1,dt1 = np.linspace(retstep =)
t2,dt2 = np.linspace(retstep =)

freq1 = 
freq2 = 

H1 = np.fft
H2 = np.fft

H3 = np.fft
H4 = np.fft

plt.stem()

plt.axvline()
plt.axvline()
plt.xlim(-1.5, 1.5)
plt.show()

plt.axvline()
plt.axvline()
plt.stem()
plt.xlim()

plt.show()


plt.stem()
plt.axvline()
plt.axvline()
plt.xlim()

plt.show()

plt.stem()

plt.axvline()
plt.axvline()
plt.xlim()

plt.show()